In [3]:
!pip install statsmodels

In [3]:
import statsmodels; print(f'statsmodels version: {statsmodels.__version__}')

statsmodels version: 0.14.5


In [11]:
# NHSRC PHC SUPPLY CHAIN - BASELINE FORECASTING + MODEL BENCHMARKING
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')

print("🔮 NHSRC PHC BASELINE FORECASTING & MODEL BENCHMARKING")
print("=" * 70)

# 1️⃣ LOAD PREPARED DATASET
print("📥 1. Loading Prepared Dataset...")

# Fix: Parse dates with dayfirst=True for DD-MM-YYYY format
df = pd.read_csv("data/forecast_ready_timeseries.csv")
df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')

print(f"   Loaded {len(df):,} records")
print(f"   Date range: {df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")
print(f"   Unique SKUs: {df['sku_id'].nunique()}")

# Check for any failed date conversions
failed_dates = df['date'].isna().sum()
if failed_dates > 0:
    print(f"   ⚠️  Warning: {failed_dates} dates failed to parse")


# 2️⃣ CREATE SKU-LEVEL TRAINING WINDOWS
print("\n📊 2. Preparing SKU-level Forecasting Windows...")
unique_skus = df["sku_id"].unique()
forecast_results = []

print(f"   Will forecast for {len(unique_skus)} SKUs")
print(f"   Each SKU has {len(df[df['sku_id'] == unique_skus[0]])} daily records")

# 3️⃣ BUILD FUNCTIONS FOR MODELS
print("\n🤖 3. Building Baseline Forecasting Models...")

def naive_forecast(series):
    """Naive model: Use last observed value as prediction"""
    return series.shift(1)

def moving_avg(series, window):
    """Moving Average: Average of last 'window' observations"""
    return series.rolling(window=window, min_periods=1).mean().shift(1)

def ets_forecast(series):
    """Exponential Smoothing (ETS): Adaptive weighting of past observations"""
    try:
        # Simple ETS model (no trend, no seasonality for baseline)
        model = ExponentialSmoothing(
            series, 
            trend=None, 
            seasonal=None,
            initialization_method='estimated'
        ).fit()
        return model.fittedvalues
    except:
        # Fallback to moving average if ETS fails
        return moving_avg(series, window=7)

def evaluate(actual, predicted, model_name="", skip_days=30):
    """Calculate MAPE with NaN handling"""
    # Align series and skip initial NaN/inf values
    aligned = pd.DataFrame({'actual': actual, 'predicted': predicted})
    aligned = aligned.dropna()
    
    # Skip first 'skip_days' to avoid initialization bias
    if len(aligned) > skip_days:
        aligned = aligned.iloc[skip_days:]
    
    # Handle zero actual values (common in healthcare data)
    # Replace zero with small value to avoid division by zero
    actual_adj = aligned['actual'].replace(0, 0.1)
    
    # Calculate MAPE
    try:
        mape = mean_absolute_percentage_error(actual_adj, aligned['predicted']) * 100
        return round(mape, 2)
    except:
        return 999.99  # High error if calculation fails

# 4️⃣ RUN FORECASTS PER SKU
print("\n📈 4. Running Baseline Forecasts for Each SKU...")
print("   SKU-by-SKU progress:")

for i, sku in enumerate(unique_skus, 1):
    # Get SKU data
    subset = df[df["sku_id"] == sku].sort_values("date")
    y = subset["units_used"].reset_index(drop=True)
    
    # Get SKU metadata for context
    sku_name = subset["sku_name"].iloc[0]
    ved_category = subset["ved_category"].iloc[0]
    fsn_category = subset["fsn_category"].iloc[0]
    
    print(f"   {i:2}. {sku} ({sku_name[:30]}...) - {ved_category}/{fsn_category}")
    
    # Generate forecasts
    models = {
        "naive": naive_forecast(y),
        "ma_7": moving_avg(y, 7),
        "ma_14": moving_avg(y, 14),
        "ma_30": moving_avg(y, 30),
        "ets": ets_forecast(y)
    }
    
    # Evaluate each model
    for name, pred in models.items():
        mape = evaluate(y, pred, model_name=name, skip_days=30)
        forecast_results.append([
            sku, 
            sku_name, 
            ved_category, 
            fsn_category,
            name, 
            mape,
            len(y),
            y.mean(),
            y.std()
        ])

print(f"\n✅ Generated {len(forecast_results)} model evaluations")

# 5️⃣ CONVERT TO LEADERBOARD
print("\n🏆 5. Creating Model Performance Leaderboard...")
results_df = pd.DataFrame(
    forecast_results, 
    columns=["sku_id", "sku_name", "ved_category", "fsn_category", 
             "model_type", "MAPE", "records", "mean_demand", "std_demand"]
)

# Sort by MAPE (lower is better)
results_df = results_df.sort_values("MAPE")

# Save baseline performance
baseline_path = "reports/model_performance_baseline.csv"
results_df.to_csv(baseline_path, index=False)
print(f"   💾 Saved: {baseline_path}")
print(f"   Records: {len(results_df)}")

# Display summary statistics
print("\n📊 MODEL PERFORMANCE SUMMARY:")
for model in ["naive", "ma_7", "ma_14", "ma_30", "ets"]:
    model_results = results_df[results_df["model_type"] == model]
    avg_mape = model_results["MAPE"].mean()
    count = len(model_results)
    print(f"   {model:6} -> Avg MAPE: {avg_mape:.1f}% ({count} SKUs)")

# 6️⃣ SELECT BEST MODEL PER SKU
print("\n🎯 6. Selecting Best Model for Each SKU...")

# Find best model (lowest MAPE) for each SKU
best_models = results_df.loc[results_df.groupby("sku_id")["MAPE"].idxmin()]

# Add confidence rating based on MAPE
def map_confidence(mape):
    if mape < 20:
        return "HIGH"
    elif mape < 40:
        return "MEDIUM"
    else:
        return "LOW"

best_models["forecast_confidence"] = best_models["MAPE"].apply(map_confidence)

# Save best model selection
best_model_path = "reports/best_model_selection.csv"
best_models.to_csv(best_model_path, index=False)
print(f"   💾 Saved: {best_model_path}")

# 7️⃣ BEST MODEL DISTRIBUTION ANALYSIS
print("\n📈 7. Best Model Distribution Analysis:")

model_distribution = best_models["model_type"].value_counts()
total_skus = len(best_models)

print("   BEST MODEL COUNTS:")
for model in ["naive", "ma_7", "ma_14", "ma_30", "ets"]:
    count = model_distribution.get(model, 0)
    percentage = (count / total_skus) * 100
    print(f"   {model:6} -> {count:2} SKUs ({percentage:.1f}%)")

# VED-category analysis
print("\n   BEST MODEL BY VED CATEGORY:")
for ved in ["Vital", "Essential", "Desirable"]:
    ved_models = best_models[best_models["ved_category"] == ved]
    if len(ved_models) > 0:
        top_model = ved_models["model_type"].mode()[0]
        avg_mape = ved_models["MAPE"].mean()
        print(f"   {ved:10} -> {top_model:6} (Avg MAPE: {avg_mape:.1f}%)")

# 8️⃣ FINAL OUTPUTS FOR TRAINER
print("\n" + "="*70)
print("🎯 TRAINER OUTPUTS")
print("="*70)

print("\n1. 🔹 FIRST 10 ROWS OF MODEL_PERFORMANCE_BASELINE.CSV:")
print("-" * 70)
baseline_preview = results_df.head(10).copy()
# Format for clean display
baseline_preview["MAPE"] = baseline_preview["MAPE"].apply(lambda x: f"{x:.1f}%")
print(baseline_preview[["sku_id", "sku_name", "model_type", "MAPE"]].to_string(index=False))

print("\n2. 🔹 BEST MODEL COUNT:")
print("-" * 70)
for model in ["naive", "ma_7", "ma_14", "ma_30", "ets"]:
    count = model_distribution.get(model, 0)
    print(f"   {model}: {count} SKUs")

print("\n3. 🔹 UPDATED GIT LS-FILES:")
print("-" * 70)
import subprocess
result = subprocess.run(['git', 'ls-files'], capture_output=True, text=True)
print(result.stdout)

print("\n" + "="*70)
print("✅ DAY 5 BASELINE FORECASTING COMPLETE")
print("="*70)
print("\n📌 Key Insights:")
print("   • Baseline models established for all 12 SKUs")
print("   • MAPE scores show forecast difficulty per medicine type")
print("   • Best model selection ready for Day 6 optimization")
print("   • Ready for NHSRC-compliant forecasting engine")

🔮 NHSRC PHC BASELINE FORECASTING & MODEL BENCHMARKING
📥 1. Loading Prepared Dataset...
   Loaded 2,160 records
   Date range: 2024-01-01 to 2024-06-28
   Unique SKUs: 12

📊 2. Preparing SKU-level Forecasting Windows...
   Will forecast for 12 SKUs
   Each SKU has 180 daily records

🤖 3. Building Baseline Forecasting Models...

📈 4. Running Baseline Forecasts for Each SKU...
   SKU-by-SKU progress:
    1. MED001 (Paracetamol Tablet 500mg...) - Vital/Fast
    2. MED002 (Amoxicillin Capsule 250mg...) - Vital/Fast
    3. MED003 (Oral Rehydration Salts...) - Vital/Fast
    4. MED004 (Ibuprofen Tablet 200mg...) - Essential/Fast
    5. MED005 (Cetirizine Tablet 5mg...) - Essential/Fast
    6. MED006 (Insulin Injection 40 IU/ml...) - Vital/Slow
    7. MED007 (Adrenaline Injection 1 mg/ml...) - Vital/Slow
    8. MED008 (Omeprazole Capsule 20mg...) - Essential/Slow
    9. MED009 (Metformin Tablet 500mg...) - Essential/Slow
   10. MED010 (Multivitamin Tablet...) - Desirable/Slow
   11. MED011 (An